In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/FTSL/1000_participants

/content/drive/.shortcut-targets-by-id/1CI1F_BNtclDb589EJiZx_ypJ_TSazyxZ/FTSL/1000_participants


In [ ]:
!pip3 install tensorflow_model_optimization

     |████████████████████████████████| 174kB 9.1MB/s 


In [ ]:
import os
import sys
import tempfile

import cv2
import keras
import numpy as np
from numpy import dstack 
import pandas as pd
from keras.layers import (Conv2D, Dense, Dropout, Flatten, GaussianNoise,
                          MaxPooling2D, MaxPool2D , Activation)
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.utils import np_utils, to_categorical
from matplotlib import pyplot
from numpy import dstack, mean, std
from pandas import read_csv
from PIL import Image
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

%matplotlib inline
import random

import matplotlib.pyplot as plt
import numpy
import plotly.express as px
import requests
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from scipy.spatial.distance import euclidean as euc

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot

init_notebook_mode(connected=True)

In [ ]:
DF_train = pd.read_csv('mnist_train.csv')
DF_test = pd.read_csv('mnist_test.csv')

In [ ]:
DF = pd.concat([DF_train, DF_test], axis = 0)
DF = DF.sample(frac=1).reset_index(drop=True)
DF

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,1x11,1x12,1x13,1x14,1x15,1x16,1x17,1x18,1x19,1x20,1x21,1x22,1x23,1x24,1x25,1x26,1x27,1x28,2x1,2x2,2x3,2x4,2x5,2x6,2x7,2x8,2x9,2x10,2x11,...,27x17,27x18,27x19,27x20,27x21,27x22,27x23,27x24,27x25,27x26,27x27,27x28,28x1,28x2,28x3,28x4,28x5,28x6,28x7,28x8,28x9,28x10,28x11,28x12,28x13,28x14,28x15,28x16,28x17,28x18,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
69996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
69997,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
69998,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
per_client_data = 3000
test_split = 0.3

C1 = DF.iloc[:per_client_data, :]
C2 = DF.iloc[per_client_data:per_client_data*2, :]
C3 = DF.iloc[per_client_data*2:per_client_data*3, :]
C4 = DF.iloc[per_client_data*3:per_client_data*4, :]
C5 = DF.iloc[per_client_data*4:per_client_data*5, :]

C1_Train = C1.iloc[:int((1-test_split)*len(C1)), :]
C2_Train = C2.iloc[:int((1-test_split)*len(C2)), :]
C3_Train = C3.iloc[:int((1-test_split)*len(C3)), :]
C4_Train = C4.iloc[:int((1-test_split)*len(C4)), :]
C5_Train = C5.iloc[:int((1-test_split)*len(C5)), :]

C1_Test = C1.iloc[int((1-test_split)*len(C1)):, :]
C2_Test = C2.iloc[int((1-test_split)*len(C2)):, :]
C3_Test = C3.iloc[int((1-test_split)*len(C3)):, :]
C4_Test = C4.iloc[int((1-test_split)*len(C4)):, :]
C5_Test = C5.iloc[int((1-test_split)*len(C5)):, :]

In [ ]:
DF = pd.concat([DF, pd.concat([C1_Train, C2_Train, C3_Train, C4_Train, C5_Train, C1_Test, C2_Test, C3_Test, C4_Test, C5_Test])]).drop_duplicates(keep=False)

Train = DF.iloc[:int((1-test_split)*len(DF)), :]
Test = DF.iloc[int((1-test_split)*len(DF)):, :]

In [ ]:
# ----------------------------- #
# ---------- SETTINGS ----------#
# ----------------------------- #

NUM_Clients = 1000 # number of clients contributing per training round

# ML
Cluster_Size = 10 # max client dataset size for training
Batch_Size = 16
NUM_Epoch = 3
verbose = 1

# Krum
krum_f = 0.25 # percentage of byzantine nodes

# Differential Privacy
Gaussian_Noise = False
Gaussian_Noise_Std_Dev = 0.20

Gradient_Clipping = False
Clip_Norm = 0.60

Gradient_Pruning = False
initial_sparsity = 0.00
final_sparsity = 0.50

# ---------------------------- #
# ----------------------------- #
# ----------------------------- #

In [ ]:
def preprocess(df):
  y = df.label
  X = df.drop(['label'], axis=1)
  X = X.values.reshape(-1,28 ,28,1)
  # print('Data size : ', X.shape)
  X = X/255.0

  y = to_categorical(y, num_classes=10)

  # print('X Shape: ', X.shape)
  # print('y Shape: ', y.shape)
  return X, y

In [ ]:
X_test, y_test = preprocess(Test)

In [ ]:
def train(name, X_train, y_train, globalId):

    n_timesteps, n_features, n_outputs = X_train.shape[0], X_train.shape[1], y_train.shape[0]

    model = Sequential()

    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(28,28,1)))

    if Gaussian_Noise == True:
        model.add(GaussianNoise(Gaussian_Noise_Std_Dev))

    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation="softmax"))#2 represent output layer neurons
    if Gradient_Pruning == True:
        end_step = np.ceil(n_timesteps / Batch_Size).astype(np.int32) * NUM_Epoch

        # Define model for pruning.
        pruning_params = {
              'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                       final_sparsity=final_sparsity,
                                                                       begin_step=0,
                                                                       end_step=end_step)
        }

        logdir = tempfile.mkdtemp()

        callbacks = [
          tfmot.sparsity.keras.UpdatePruningStep(),
          tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
        ]

        model = prune_low_magnitude(model, **pruning_params)

    model.built = True

    if globalId != 1:
        model.load_weights("./weights/global"+str(globalId)+".h5")

    if Gradient_Clipping == True:
        opt = keras.optimizers.Adam(clipnorm=Clip_Norm)
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    else: 
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    if Gradient_Pruning == True:
        history = model.fit(X_train, y_train, batch_size=Batch_Size, epochs=NUM_Epoch, verbose=1, callbacks=callbacks)
    else:
        history = model.fit(X_train, y_train, batch_size=Batch_Size, epochs=NUM_Epoch, verbose=1)

    #Saving Model
    model.save("./weights/"+str(name)+".h5")
    return n_timesteps, model

In [ ]:
def euclidean(m, n):
    # Finds eucledian distance between two ML models m & n
    distance = []
    for i in range(len(m)):
        distance.append(euc(m[i].reshape(-1,1), n[i].reshape(-1,1)))
    distance = sum(distance)/len(m)
    return distance

def saveModel(weight, n):
    
    num_classes=len(np.unique(y_test))

    model = Sequential()

    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(28,28,1)))

    if Gaussian_Noise == True:
        model.add(GaussianNoise(Gaussian_Noise_Std_Dev))

    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation="softmax"))#2 represent output layer neurons

    if Gradient_Pruning == True:
          end_step = np.ceil(16500 / Batch_Size).astype(np.int32) * NUM_Epoch

          # Define model for pruning.
          pruning_params = {
                'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                        final_sparsity=final_sparsity,
                                                                        begin_step=0,
                                                                        end_step=end_step)
          }

          logdir = tempfile.mkdtemp()

          callbacks = [
            tfmot.sparsity.keras.UpdatePruningStep(),
            tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
          ]

          model = prune_low_magnitude(model, **pruning_params)

    model.set_weights(weight)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    scores = model.evaluate(X_test, y_test)
    print("Loss: ", scores[0])        #Loss
    print("Accuracy: ", scores[1])    #Accuracy

    #Saving Model
    fpath = "./weights/global"+str(n)+".h5"
    model.save(fpath)
    return scores[0], scores[1]

def getDataLen(trainingDict):
    n = 0
    for w in trainingDict:
        n += trainingDict[w]
    print('Total number of data points after this round: ', n)
    return n

def assignWeights(trainingDf, trainingDict):
    n = getDataLen(trainingDict)
    trainingDf['Weightage'] = trainingDf['DataSize'].apply(lambda x: x/n)
    return trainingDf, n
    
def scale(weight, scaler):
    scaledWeights = []
    for i in range(len(weight)):
        scaledWeights.append(scaler * weight[i])
    return scaledWeights

def getScaledWeight(d, scaler):
    model = Sequential()

    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(28,28,1)))

    if Gaussian_Noise == True:
        model.add(GaussianNoise(Gaussian_Noise_Std_Dev))

    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation="softmax"))#2 represent output layer neurons

    if Gradient_Pruning == True:
        model = prune_low_magnitude(model)
    fpath = "./weights/"+d+".h5"
    model.load_weights(fpath)
    weight = model.get_weights()
    return scale(weight, scaler)

def getWeight(d):
    model = Sequential()

    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(28,28,1)))

    if Gaussian_Noise == True:
        model.add(GaussianNoise(Gaussian_Noise_Std_Dev))

    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation="softmax"))#2 represent output layer neurons

    if Gradient_Pruning == True:
        model = prune_low_magnitude(model)
    fpath = "./weights/"+d+".h5"
    model.load_weights(fpath)
    weight = model.get_weights()
    return weight

def avgWeights(scaledWeights):
    avg = list()
    for weight_list_tuple in zip(*scaledWeights):
        layer_mean = tf.math.reduce_sum(weight_list_tuple, axis=0)
        avg.append(layer_mean)
    return avg

def FedAvg(trainingDict):
    trainingDf = pd.DataFrame.from_dict(trainingDict, orient='index', columns=['DataSize']) 
    models = list(trainingDict.keys())
    scaledWeights = []
    trainingDf, dataLen = assignWeights(trainingDf, trainingDict)
    for m in models:
        scaledWeights.append(getScaledWeight(m, trainingDf.loc[m]['Weightage']))
    fedAvgWeight = avgWeights(scaledWeights)
    return fedAvgWeight, dataLen

def MK(trainingDict, b):
    models = list(trainingDict.keys())
    trainingDf = pd.DataFrame.from_dict(trainingDict, orient='index', columns=['DataSize'])
    l_weights = []
    g_weight = {}
    for m in models:
        if 'global' in m:
            g_weight['name'] = m
            g_weight['weight'] = getWeight(m)
        else:
            l_weights.append({
                'name': m,
                'weight': getWeight(m)
            })
    scores = {}
    for m in l_weights:
        scores[m['name']] = euclidean(m['weight'], g_weight['weight'])
    sortedScores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1])}

    b = int(len(scores)*b)
    
    selected = []
    for i in range(b):
        selected.append((sortedScores.popitem())[0])

    newDict = {}
    for i in trainingDict.keys():
        if (((i not in selected) and ('global' not in i))):
            newDict[i] = trainingDict[i]

    print('Selections: ', newDict)
    NewGlobal, dataLen = FedAvg(newDict)
    return NewGlobal, dataLen


In [ ]:
curr_local = 0
curr_global = 0
def fed():
  global curr_local
  global curr_global

  local = {}
  loss_array = []
  acc_array = []
  for i in range(0, len(Train), Cluster_Size):

      if int(curr_global) == 0:
        curr_global += 1
        print('Current Global: ', curr_global)
        name = 'global' + str(curr_global)
        X_train, y_train = preprocess(Train[i:i+Cluster_Size])
        l, m = train(name, X_train, y_train, curr_global)
        local[name] = l

      else:
        print('Current Local: ', curr_local)
        name = str('local'+str(curr_local))
        curr_local += 1
        X_train, y_train = preprocess(Train[i:i+Cluster_Size])
        if X_train.shape[0]<=Cluster_Size-1:
          continue
        l, m = train(name, X_train, y_train, curr_global)
        local[name] = l

        if (int(curr_local)%NUM_Clients == 0) and (curr_local != 0):
          curr_global += 1
          print('Current Global: ', curr_global)
          name = 'global' + str(curr_global)
          m, l = MK(local, krum_f)
          loss, acc = saveModel(m, curr_global)
          loss_array.append(loss)
          acc_array.append(acc)
          local = {}
          local[name] = l
  return acc_array, loss_array

In [ ]:
fed()

Streaming output truncated to the last 5000 lines.
Current Local:  3224
Epoch 1/3
1/1 [==============================] - 1s 803ms/step - loss: 2.3026 - accuracy: 0.1000
Epoch 2/3
1/1 [==============================] - 0s 7ms/step - loss: 2.3020 - accuracy: 0.3000
Epoch 3/3
1/1 [==============================] - 0s 6ms/step - loss: 2.3012 - accuracy: 0.0000e+00
Current Local:  3225
Epoch 1/3
1/1 [==============================] - 1s 587ms/step - loss: 2.3025 - accuracy: 0.2000
Epoch 2/3
1/1 [==============================] - 0s 22ms/step - loss: 2.3013 - accuracy: 0.2000
Epoch 3/3
1/1 [==============================] - 0s 6ms/step - loss: 2.2996 - accuracy: 0.2000
Current Local:  3226
Epoch 1/3
1/1 [==============================] - 1s 511ms/step - loss: 2.3027 - accuracy: 0.1000
Epoch 2/3
1/1 [==============================] - 0s 7ms/step - loss: 2.3017 - accuracy: 0.3000
Epoch 3/3
1/1 [==============================] - 0s 7ms/step - loss: 2.3005 - accuracy: 0.3000
Current Local:  3227

([0.1156969666481018, 0.10515151172876358, 0.1156969666481018],
 [2.302574872970581, 2.30257248878479, 2.302577018737793])

In [ ]:
# 50,50 ->>  [0.11212120950222015,
#   0.11212120950222015,
#   0.13884848356246948,
#   0.15187878906726837,
#   0.2223636358976364,
#   0.21460606157779694,
#   0.2707878649234772,
#   0.38206061720848083,
#   0.4484848380088806,
#   0.4824848473072052,
#   0.5224848389625549,
#   0.545575737953186,
#   0.5599393844604492,
#   0.5815151333808899,
#   0.608848512172699]

In [ ]:
def getNoTL(X_test, y_test, curr_global):
    num_classes=len(np.unique(y_test))

    model = Sequential()

    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(28,28,1)))

    if Gaussian_Noise == True:
        model.add(GaussianNoise(Gaussian_Noise_Std_Dev))

    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation="softmax"))#2 represent output layer neurons

    if Gradient_Pruning == True:
          end_step = np.ceil(16500 / Batch_Size).astype(np.int32) * NUM_Epoch

          # Define model for pruning.
          pruning_params = {
                'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                        final_sparsity=final_sparsity,
                                                                        begin_step=0,
                                                                        end_step=end_step)
          }

          logdir = tempfile.mkdtemp()

          callbacks = [
            tfmot.sparsity.keras.UpdatePruningStep(),
            tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
          ]

          model = prune_low_magnitude(model, **pruning_params)

    model.load_weights("./weights/global"+str(curr_global)+".h5")

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
      
    scores = model.evaluate(X_test, y_test)
    print("NoFed Loss: ", scores[0])        #Loss
    print("NoFed Accuracy: ", scores[1])    #Accuracy

In [ ]:
def getNoFed(X_train, y_train, curr_global):

    X_test, y_test = preprocess(Test)

    num_classes=len(np.unique(y_test))

    model = Sequential()

    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(28,28,1)))

    if Gaussian_Noise == True:
        model.add(GaussianNoise(Gaussian_Noise_Std_Dev))

    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation="softmax"))#2 represent output layer neurons

    if Gradient_Pruning == True:
          end_step = np.ceil(16500 / Batch_Size).astype(np.int32) * NUM_Epoch

          # Define model for pruning.
          pruning_params = {
                'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                        final_sparsity=final_sparsity,
                                                                        begin_step=0,
                                                                        end_step=end_step)
          }

          logdir = tempfile.mkdtemp()

          callbacks = [
            tfmot.sparsity.keras.UpdatePruningStep(),
            tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
          ]

          model = prune_low_magnitude(model, **pruning_params)

    # model.load_weights("./weights/global"+str(curr_global)+".h5")

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    if Gradient_Pruning == True:
        history = model.fit(X_train, y_train, batch_size=Batch_Size, epochs=NUM_Epoch, verbose=1, callbacks=callbacks)
    else:
        history = model.fit(X_train, y_train, batch_size=Batch_Size, epochs=NUM_Epoch, verbose=1)
      
    scores = model.evaluate(X_test, y_test)
    print("Centralised Loss: ", scores[0])        #Loss
    print("Centralised Accuracy: ", scores[1])    #Accuracy

In [ ]:
def TransferLearn(name, traindf, testdf):
  global curr_global
  X_train, y_train = preprocess(traindf)
  X_test, y_test = preprocess(testdf)

  n_timesteps, n_features, n_outputs = X_train.shape[0], X_train.shape[1], y_train.shape[0]

  inner_model = Sequential(
    [
        Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(28,28,1))
    ]
)

  model = Sequential()
  model.add(inner_model)
  
  if Gaussian_Noise == True:
      model.add(GaussianNoise(Gaussian_Noise_Std_Dev))

  model.add(MaxPooling2D(pool_size=2))
  model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
  model.add(MaxPooling2D(pool_size=2))
  model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
  model.add(MaxPooling2D(pool_size=2))
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(500,activation="relu"))
  model.add(Dropout(0.2))
  model.add(Dense(10,activation="softmax"))#2 represent output layer neurons


  if Gradient_Pruning == True:
        end_step = np.ceil(n_timesteps / Batch_Size).astype(np.int32) * NUM_Epoch

        # Define model for pruning.
        pruning_params = {
              'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                       final_sparsity=final_sparsity,
                                                                       begin_step=0,
                                                                       end_step=end_step)
        }

        logdir = tempfile.mkdtemp()

        callbacks = [
          tfmot.sparsity.keras.UpdatePruningStep(),
          tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
        ]

        model = prune_low_magnitude(model, **pruning_params)

  model.load_weights("./weights/global"+str(curr_global)+".h5")

  if Gradient_Clipping == True:
    opt = keras.optimizers.Adam(clipnorm=Clip_Norm)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  else: 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  for layer in inner_model.layers:#freezing layers to retain the weights 
	  layer.trainable = False
      
  if Gradient_Pruning == True:
    history = model.fit(X_train, y_train, epochs=15, batch_size=Batch_Size, verbose=1, callbacks=callbacks)
  else:
    history = model.fit(X_train, y_train, epochs=15, batch_size=Batch_Size, verbose=1)

  history2 = model.evaluate(X_test, y_test, batch_size=Batch_Size, verbose=1)

  test_accuracy = history2[1]
  print(f"The test accuracy after Transfer Learning is {test_accuracy}")

  nofedAcc = getNoTL(X_test, y_test, curr_global)

  getCent = getNoFed(X_train, y_train, curr_global)
  
  #Saving Model
  model.save("./weights/"+str(name)+".h5")
  return n_timesteps, model

In [ ]:
TransferLearn('C1', C1_Train, C1_Test)

In [ ]:
TransferLearn('C2', C2_Train, C2_Test)

In [ ]:
TransferLearn('C3', C3_Train, C3_Test)

In [ ]:
TransferLearn('C4', C4_Train, C4_Test)

In [ ]:
TransferLearn('C5', C5_Train, C5_Test)

In [ ]:
!ls